<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/Module4_LASSOandCaravan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Regularized Regression

In this tutorial, we will discuss regularized regression approaches, particularly least absolute shrinkage and selection operator -- or, in short, the [LASSO](http://statweb.stanford.edu/~tibs/lasso.html/) -- in the context of predicting claim sizes.

Let's install relevant packages. Again, we're going to rely on the statistical learning toolkit ski-cit learn, which provides LASSO regression but also has many other predictive models. As before, it is less comfortable to use than some of the other packages and, unlike R, does not support formulas. But it is versatile and fast, and therefore one of the most popular predictive modeling toolkits.

In [1]:
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier

## Ridge and LASSO Regression

Ridge and LASSO regression are both examples of *regularized* regression approaches.  In what follows, we will first briefly review the corresponding approaches, and particularly highlight how they differ from their unregularized counterparts.   We then will work through a simulated example to become familiar with the impact of the *tuning parameter* on the resulting coefficient estimates.  We will also determine the in- and out-of-sample fit depending on the choice of the tuning parameter, uncovering a familiar relationship.

## Review of Concepts and Maths

In a conventional (linear) regression problem with independent variables $x_i$ and depedent variables $y_i$, we are determining the best fit in the least-squares sense:
$$
\hat{\beta}^{\text{OLS}} = \text{argmin}_{\beta}\left\{\sum_{i=1}^n \left(y_i - \left(\beta_0 + \sum_{j=1}^p \beta_j\,x_{i,j}\right)\right)^2 \right\}.
$$
Within a *regularized* approach, we now include penalties for choosing many or large parameters:
$$
\hat{\beta}^{\text{REG}}_\lambda = \text{argmin}_{\beta}\left\{\sum_{i=1}^n \left(y_i - \left(\beta_0 + \sum_{j=1}^p \beta_j\,x_{i,j}\right)\right)^2 + \lambda \times R(\beta) \right\}.
$$
Here, $R(\beta)$ is a so-called *regularization* term that imposes a penalty on the complexity of the regression equation.  In particular, within Rigde regression the penalty term is *quadratic*, $R(\beta) = \sum_{j=1}^p \beta_j^2,$ wheras the LASSO uses an L1 penalty, $R(\beta) = \sum_{j=1}^p |\beta_j|.$  

We call $\lambda$ the *tuning parameter*, and it governs how sizable the complexity penalty will be.  In particular, note that for $\lambda=0$ we are back to the unregularized problem, whereas for large lambda the penalty will be severe -- so this will lead to *shrinkage* of the coefficient estimates.  As $\lambda$ becomes large and larger, the prediction will more and more closely resemble a *constant* prediction, $\hat{y}_i = \beta_0.$  Thus, the choice of the tuning parameter will directly be related to trading off a reduction in variance (due to shrinkage) with an increase in bias (due to the less flexible model fit).  Again, we will explore these aspects in more detail in the context of our example below.


# Caravan Insurance Application

We look at the `Caravan` insurance data set included in the ISLR textbook. As indicated in Section 4.6.6, it is a dataset that includes 85 predictors that measure demographic characteristics for 5,822 individuals and "Purchase," which indicates whether or not a given individual purchases a caravan insurance policy. 

As usual, let's load some relevant libraries:

Let's load our dataset:

In [ ]:
!git clone https://github.com/danielbauer1979/ML_656.git

In [3]:
Caravan = pd.read_csv('ML_656/Caravan.csv', index_col=0) 

## Some Data Exploration

In [ ]:
Caravan.head()

Variables 1-43 represent sociodemographic data, variables 44-86 describe product ownership, and Variable 86 (Purchase) indicates whether the customer purchased a caravan insurance policy.

Let's consider some aggregate statistics:

In [ ]:
Caravan.describe()

And check how many people purchase insurance:

In [ ]:
Caravan['Purchase'].value_counts()

So only roughly 6% of all people buy caravan insurance.  That will be costly for an insurance agent because for every client she or he visits, only 6 in 100 will purchase insurance.  So let's use our knowledge about classification to help out the sales force, and let's try to determine individuals (based on their characteristics) that are more likely to purchase a policy.

## Predictive Modeling

Let's split into a training and test set to get going

In [7]:
Train, Test = train_test_split(Caravan, test_size=0.25, random_state=1)

In [8]:
X_train = Train.drop(['Purchase'], axis=1)
y_train = Train['Purchase']
X_test = Test.drop(['Purchase'], axis=1)
y_test = Test['Purchase']

Let's start with a vanilla logistic regression model:

In [9]:
logistic_model = LogisticRegression(fit_intercept=True, max_iter=1000)
logistic_model.fit(X_train,y_train)
y_pred_logistic = logistic_model.predict(X_test)

Let's look at the confusion matrix resulting from our predictions (here the predicted probabilities are already coerced to classes):

In [ ]:
confusion_matrix(y_test, y_pred_logistic)

We don't get a single positive one right -- so not great performance.  Of course, we could choose a different cutoff.  Let's evaluate the AUC, where we first have to convert the predictions to probabilities:

In [11]:
y_pred_logistic = logistic_model.predict_proba(X_test)
def Extract(lst): 
    return [item[0] for item in lst] 
y_pred_logistic = Extract(y_pred_logistic)

In [ ]:
fpr, tpr, threshold = roc_curve((Test['Purchase'] == 'No'), y_pred_logistic) 
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic') 
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc) 
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--') 
plt.xlim([0, 1])
plt.ylim([0, 1]) 
plt.ylabel('True Positive Rate') 
plt.xlabel('False Positive Rate') 
plt.show()

Let's check LDA:

In [ ]:
LDA_model = LinearDiscriminantAnalysis()
LDA_model.fit(X_train,y_train)
y_pred_LDA = LDA_model.predict(X_test)
np.sum(y_pred_LDA == 'Yes')

So it looks like we get a few positives. Let's check confusion matrix and AUC:

In [ ]:
confusion_matrix(y_test, y_pred_LDA)

In [15]:
y_pred_LDA = LDA_model.predict_proba(X_test) 
y_pred_LDA = Extract(y_pred_LDA)

In [ ]:
fpr, tpr, threshold = roc_curve((Test['Purchase'] == 'No'), y_pred_LDA) 
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic') 
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc) 
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--') 
plt.xlim([0, 1])
plt.ylim([0, 1]) 
plt.ylabel('True Positive Rate') 
plt.xlabel('False Positive Rate') 
plt.show()

So a bit of an improvement.

Let's compare these results to an L1-regularized logistic regression -- a.k.a. LASSO logistic regression -- to see if that yields an improvement.  After all, there are many features so possibily selection is important:

In [17]:
lassolog_model = LogisticRegression(
    penalty='l1',
    solver='saga',
    max_iter=2000)  # or 'liblinear'
lassolog_model.fit(X_train, y_train)
y_pred_lassolog = lassolog_model.predict(X_test)

Let's evaluate the predictions:

In [ ]:
confusion_matrix(y_test, y_pred_lassolog)

Let's try to tune the model a bit better (this part takes a bit, so you can skip):

In [ ]:
C = [50, 10, 1, .1, 0.05,.01,.001]
for c in C:
  lassologcv_model = LogisticRegression(penalty='l1',C=c,class_weight = 'balanced',solver='liblinear',max_iter=2000)
  scores = cross_val_score(lassologcv_model, X_train, y_train, cv=5, scoring="f1_micro")  
  print(scores)
  print(np.mean(scores))

Let's evaluate predictions:

In [ ]:
lassologcv_model = LogisticRegression(penalty='l1',C=50,class_weight = 'balanced',solver='liblinear',max_iter=2000)
lassologcv_model.fit(X_train, y_train)
y_pred_lassologcv = lassologcv_model.predict(X_test)
confusion_matrix(y_test, y_pred_lassologcv)

So at least we find some positives. And let's check the AUC again:

In [ ]:
y_pred_lassologcv = lassologcv_model.predict_proba(X_test)
y_pred_lassologcv = Extract(y_pred_lassologcv)

In [ ]:
fpr, tpr, threshold = roc_curve((Test['Purchase'] == 'No'), y_pred_lassologcv) 
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic') 
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc) 
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--') 
plt.xlim([0, 1])
plt.ylim([0, 1]) 
plt.ylabel('True Positive Rate') 
plt.xlabel('False Positive Rate') 
plt.show()

No substantial improvement. So it looks like here selecting variables doesn't make a huge difference. Possibly more advanced learners that can spot relevant interactions will perform better.